In [1]:
select version();

version()
10.7.0-MariaDB-MDEV-25015-sformat


# Create test database and table

In [2]:
create database preview_sformat;

In [2]:
use preview_sformat;

In [2]:
create table t1(rel char(6), mdev int, feature char(20));

In [2]:
insert into t1 values('10.7.0', 25015, 'Python style sformat'), 
                     ('10.7.0', 4958, 'UUID');

In [2]:
select * from t1;

rel,mdev,feature
10.7.0,25015,Python style sformat
10.7.0,4958,UUID


# Show simple sformat usage

In [3]:
select sformat('MariaDB Server {} has a preview for MDEV-{} which is about {}', 
       rel, mdev, feature) as 'Preview Release Examples' from t1;

Preview Release Examples
MariaDB Server 10.7.0 has a preview for MDEV-25015 which is about Python style sformat
MariaDB Server 10.7.0 has a preview for MDEV-4958 which is about UUID


# Test case with some intentional bugs

In [11]:
create or replace table t1 (param1 text, param2 text, param3 int);

Query OK

In [12]:
insert into  t1 values ('string: {}, number: {}', 'xyz', 123),
                       ('something {} went {} wrong {}', 'foo', 0),
                       ('Test case {} -> {}', 'Print', -32);

Query OK

In [13]:
select sformat(param1, param2, param3) from t1;

"sformat(param1, param2, param3)"
"string: xyz, number: 123"
NULL
Test case Print -> -32


# FLOATs and DATEs

In [7]:
create or replace table t2 (param1 FLOAT, param2 SMALLINT, param3 CHAR, param4 DATE);

Query OK

In [8]:
insert into t2 values (0.0025, 25, 'A', DATE('2020-06-29')),
                      (0.0005,  5, 'B', DATE('2020-6-29')),
                      (5.5555, -5, 'C', DATE('200629')),
                      (-9,     -9, 'D', DATE('20*06*29'));

Query OK

In [9]:
select sformat('p1 {:.4f} p2 {} p3 {} p4 {}', param1, param2, param3, param4) from t2;

"sformat('p1 {:.4f} p2 {} p3 {} p4 {}', param1, param2, param3, param4)"
p1 0.0025 p2 25 p3 A p4 2020-06-29
p1 0.0005 p2 5 p3 B p4 2020-06-29
p1 5.5555 p2 -5 p3 C p4 2020-06-29
p1 -9.0000 p2 -9 p3 D p4 2020-06-29


# utf8 vs. latin1

In [14]:
set names utf8;

Query OK

In [17]:
create or replace table t3 (format_str   text character set 'latin1',
                            first_param  text character set 'utf8',
                            second_param text character set 'latin1');

Query OK

In [18]:
insert into t3 values ('test 1 {} {}', UNHEX('C3A5'), UNHEX('E5'));

Query OK

In [19]:
select sformat(format_str, first_param, second_param) from t3;

"sformat(format_str, first_param, second_param)"
test 1 å å


In [20]:
select HEX(sformat(format_str, first_param, second_param)) from t3;

"HEX(sformat(format_str, first_param, second_param))"
74657374203120C3A520C3A5


# bit, boolean, decimal, timestamp

In [21]:
set names latin1;

Query OK

In [22]:
create or replace table t4 (p1 bit(8), p2 boolean, p3 DECIMAL, p4 TIMESTAMP);

Query OK

In [23]:
insert into t4 values (42, TRUE,  42, '2021-08-18 16:50:07'),
                      (24, FALSE, 24, '0000-00-00 00:00:00');

Query OK

In [24]:
select sformat('{}: {} {} {} {}', 'Data', p1, p2, p3, p4) from t4;

"sformat('{}: {} {} {} {}', 'Data', p1, p2, p3, p4)"
Data: 42 1 42 2021-08-18 16:50:07
Data: 24 0 24 0000-00-00 00:00:00


# HEX and more utf8

In [25]:
set names utf8;

Query OK

In [26]:
create or replace table t5 (param text character SET utf8 COLLATE utf8_general_ci);

Query OK

In [27]:
insert into t5 values (UNHEX('C3A5')),(UNHEX('C5BB')),(UNHEX('e0b1bb')),
                      (UNHEX('C38A')), (NULL);

Query OK

In [28]:
select sformat('{}', param) from t5;

"sformat('{}', param)"
å
Ż
౻
Ê
NULL


In [29]:
select HEX(sformat('{}', param)) from t5;

"HEX(sformat('{}', param))"
C3A5
C5BB
E0B1BB
C38A
NULL


# A large number of sformat examples

In [22]:
select sformat('Num {:L}', 13800000000);

In [22]:
select sformat('Num [{:20}]', 42);

In [22]:
select sformat('Number: {:*^{}}', 4, 5);

In [22]:
select sformat('{:02} - {:02} - {:02}', 1, 2, 3);

In [22]:
select sformat('Character {:c}', 'h');

In [22]:
select sformat('String {:s}', 'hello');

In [22]:
select sformat('Large {0:+010.4g}', 392.64);

In [22]:
select sformat('Large {:g}', 392.65);

In [22]:
select sformat('Float {:.2f}', 42.0);

In [22]:
select sformat('Float {:f}', 42.0);

In [22]:
select sformat('Number {:d}', 42);

In [22]:
select sformat('Number {:{}}', 5, 5);

In [22]:
select sformat('Number [{:10}]', 9999);

In [22]:
select sformat('Number {:.3}', 3.1416);

In [22]:
select sformat('int: {0:d};  hex: {0:x};  oct: {0:o}', 42);

In [22]:
select sformat('int: {0:d};  hex: {0:#x};  oct: {0:#o}', 42);

In [22]:
select sformat('The hexadecimal version of {0} is {0:X}', 255);

In [22]:
select sformat('The hexadecimal version of {0} is {0:x}', 255);

In [22]:
select sformat('The octal version of {0} is {0:o}', 10);

In [22]:
select sformat('The binary version of {0} is {0:b}', 5);

In [22]:
select sformat('{:+f}; {:+f}', 3.14, -3.14);

In [22]:
select sformat('{: f}; {: f}', 3.14, -3.14);

In [22]:
select sformat('{:-f}; {:-f}', 3.14, -3.14);

In [22]:
select sformat('The temperature is between {: } and {: } degrees celsius.', -3, 7);

In [22]:
select sformat('The temperature is between {:-} and {:-} degrees celsius.', -3, 7);

In [22]:
select sformat('The temperature is between {:+} and {:+} degrees celsius.', -3, 7);

In [22]:
select sformat('We have {:<8} chickens.', 49);

In [22]:
select sformat('Center align [{:*^10}]', 'data');

In [22]:
select sformat('Center align [{:^10}].', 'data');

In [22]:
select sformat('Right align [{:>10}].', 'data');

In [22]:
select sformat('Left align [{:<10}].', 'data');

In [22]:
select sformat('{0}{1}{0}', 'abra', 'cad');

In [22]:
select sformat('Change Order {1} {0}', 'second', 'first');

"sformat('Num {:L}', 13800000000)"
"Num 13,800,000,000"
"sformat('Num [{:20}]', 42)"
Num [ 42]
"sformat('Number: {:*^{}}', 4, 5)"
Number: **4**
"sformat('{:02} - {:02} - {:02}', 1, 2, 3)"
01 - 02 - 03
"sformat('Character {:c}', 'h')"
Character h
"sformat('String {:s}', 'hello')"
